In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content/'
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

'kaggle'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [2]:
!unzip -q /content/dogs-vs-cats-redux-kernels-edition.zip -d .

'unzip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
!unzip -q /content/train.zip -d .

'unzip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [5]:
import os
import tensorflow as tf
import shutil


os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

for i in os.listdir('/content/train/'):
  if 'cat' in i: 
    shutil.copyfile('/content/train/'+i, '/content/dataset/cat/'+i)
  if 'dog' in i:
    shutil.copyfile('/content/train/'+i, '/content/dataset/dog/'+i)

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: '/content/dataset'

In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),
  batch_size=64,
  subset='training',
  validation_split=0.2,
  seed=1234
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(150,150),
  batch_size=64,
  subset='validation',
  validation_split=0.2,
  seed=1234
)

print(train_ds)

def 전처리함수(i, 정답):
  i = tf.cast( i/255.0, tf.float32 )
  return i, 정답

train_ds = train_ds.map(전처리함수)
val_ds = val_ds.map(전처리함수)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<BatchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [7]:
import requests

url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
r = requests.get(url, allow_redirects=True)

#가중치 파일
open('inception_v3.h5', 'wb').write(r.content)

87910968

In [8]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

inception_model=InceptionV3(input_shape=(150,150,3), include_top=False, weights=None)#마지막층 출력층 레이러 가져올지 말지 
inception_model.load_weights('inception_v3.h5')
inception_model.summary()

for i in inception_model.layers:
  i.trainalbe=False #가중치 고정

#fine tuning 예를들어 인셉션 모델을 mixed 6부터 트레이닝 조금 하고싶다면?
#mixded6부터 트레이닝 쭉할려면?
unfreeze=False
for i in inception_model.layers:
  if i.name=='mixed6':
    unfreeze=True
  if unfreeze ==True:
    i.trainable = True
lastlayer=inception_model.get_layer('mixed7')
print(lastlayer)

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 74, 74, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 74, 74, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                       

In [11]:
import tensorflow as tf
layer1=tf.keras.layers.Flatten()(lastlayer.output)
layer2=tf.keras.layers.Dense(1024,activation='relu')(layer1)
drop1=tf.keras.layers.Dropout(0.2)(layer2)
layer3=tf.keras.layers.Dense(1,activation='sigmoid')(drop1)

model=tf.keras.Model(inception_model.input,layer3)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=4)

Epoch 1/4
313/313 [==============================] - 98s 299ms/step - loss: 0.1009 - accuracy: 0.9604 - val_loss: 0.8862 - val_accuracy: 0.8140
Epoch 2/4
313/313 [==============================] - 92s 294ms/step - loss: 0.0704 - accuracy: 0.9724 - val_loss: 0.3346 - val_accuracy: 0.8910
Epoch 3/4
313/313 [==============================] - 83s 263ms/step - loss: 0.0523 - accuracy: 0.9807 - val_loss: 0.3714 - val_accuracy: 0.9158
Epoch 4/4
313/313 [==============================] - 97s 309ms/step - loss: 0.0441 - accuracy: 0.9834 - val_loss: 0.1701 - val_accuracy: 0.9352


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

maker=ImageDataGenerator(
    rescale=1./255,
    rotation_range=20, #회전
    zoom_range=0.15, #확대
    width_shift_range=0.2, #이동
    height_shift_range=0.2, #이동
    shear_range=0.15, # 굴절 
    horizontal_flip=True, # 가로반전
    fill_mode='nearest')

트레이닝용=maker.flow_from_directory(
    '/content/dataset',
    class_mode='binary', # 2개면 바이너리 몇개더있으면 categorical
    shuffle=True,
    seed=123,
    color_mode='rgb',
    batch_size=64,
    target_size=(64,64),
)

non_aug=ImageDataGenerator(rescale=1./255)

검증용=non_aug.flow_from_directory(
    '/content/dataset',
    class_mode='binary', # 2개면 바이너리 몇개더있으면 categorical
    shuffle=True,
    seed=123,
    color_mode='rgb',
    batch_size=64,
    target_size=(64,64),
)


#relu 쓰는 이유? 이미지에서는 숫자값은 0~255 이기떄문에 음수를 뽑을 이유가 없음
model=tf.keras.Sequential([   
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal',  input_shape=(64,64,3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),
    #컨벌루션 2d 는 4차원 데이터가 필요(a,v,c,d) 칼라 데이터라 64 64 3=(R,G,B)
    tf.keras.layers.Conv2D(32,(3,3), padding="same", activation='relu' ),
    # 이미지 가운데로 모으기(관습) pooling size(2x2)
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3), padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128,(3,3), padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    # 2차원 배열을 1차원 배열로 변환 해줌
     tf.keras.layers.Flatten(),
    #(32개의 다른 특징점을 뽑아낸 그림 추출 (3,3)의 커널 사이즈) 이과정을 진행하면 그림이 작하지는데 이때 paddig을 넣어서 픽셀을 키운다.
    #tf.keras.layers.Dense(128,input_shape=(28,28),activation="relu"),#음수는 다 0으로 만들어라 convolution layer 에서 자주씀
    tf.keras.layers.Dense(128, activation="relu"),
    #overfitin 완화 기능 
    tf.keras.layers.Dropout(0.2),
     #확률 예측(0.2 0.3 등등) sigmoid 는 0~1로 압축 하고 싶을때만(마지막 노드가 1개일때 사용)
    # softmax 는 마지막 노드수의 개수에 대한 각각의 확률 을 나타내주고 그값들의 합은 1
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

#loss 함수에서 sparse_categorical_crossentropy 은 카테고리 예측문제에서 사용하는 것
#categorical_crossentropy trainy가 원핫인코딩 되어있을때 또는 (0 1 2 3 등등으로 정수형으로 설정 되어있을때)
#원핫 인코딩
#

#요약 보고 싶으면 input_shape 넣기
model.summary()

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(트레이닝용, epochs=1)

Found 25000 images belonging to 2 classes.
Found 25000 images belonging to 2 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 64, 64, 3)         0         
                                                                 
 random_rotation (RandomRota  (None, 64, 64, 3)        0         
 tion)                                                           
                                                                 
 random_zoom (RandomZoom)    (None, 64, 64, 3)         0         
                                                                 
 conv2d_94 (Conv2D)          (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                    

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.00001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=2)

In [12]:
model.save('새폴더/model_개고양이')

INFO:tensorflow:Assets written to: 새폴더/model_개고양이\assets


INFO:tensorflow:Assets written to: 새폴더/model_개고양이\assets
